In [ ]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier






In [ ]:
# Load Data
# Shapes should be x: (n_samples, n_features), y: (n_samples, 1)

In [ ]:
#
# train decision tree
#

# options
max_depth = 5

# train - look up sklearn documentation for DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=max_depth)
dt.fit(x, y)

# print tree
tree = sklearn.tree.export_text(dt, feature_names=f)
print(tree)